## Extração, transformação e carregamento dos dados

Notebook para extração, transformação e carregamento dos dados que serão utilizados no APP Streamlit.

In [29]:
from utils import SimpleAPIClient
from utils.proj_types import JSONType
from config import SOF_TOKEN

from typing import List

import pandas as pd

In [30]:
ANO_BASE=2024

In [31]:
def get_key_dados(resp_sof:JSONType)->str:

    #obtem a chave de dados, que varia
    chaves = set(resp_sof.keys())
    chaves.discard("metaDados")
    chave_dados = chaves.pop()

    return chave_dados

In [32]:
def get_sof(endpoint:str, pagination_parameter:str='numPagina', **params)->JSONType:

    BASE_URL = 'https://gateway.apilib.prefeitura.sp.gov.br/sf/sof/v4/'

    client = SimpleAPIClient(BASE_URL)

    headers: dict[str, str] = {'Authorization' : f'Bearer {SOF_TOKEN}'}
    
    page = 1
    resp =  client.get(endpoint, headers=headers, pagination_parameter=1, **params)

    chave_dados = get_key_dados(resp)
    total_pages= resp['metaDados']['qtdPaginas']
    page+=1

    while page < total_pages:
        page_resp = client.get(endpoint, headers=headers, pagination_parameter=page, **params)
        resp[chave_dados].extend(page_resp[chave_dados])
    
    return resp

In [33]:
def clean_cnpj(cnpj:str)->str:

    cnpj = ''.join([char for char in cnpj if char.isdigit()])

    return cnpj

In [34]:
def busca_credor(cnpj:str, ano=ANO_BASE)->JSONType:
    
    cnpj = clean_cnpj(cnpj)
    endpoint = 'credores'

    return get_sof(endpoint, numCpfCnpj=cnpj)




In [35]:
from typing import Any, Dict, List


CNPJ_PRODAM = '43.076.702/0001-61'

credores: JSONType  = busca_credor(CNPJ_PRODAM)

In [36]:
prodam = credores[get_key_dados(credores)][0]

In [37]:
prodam

{'txtNomeFantasia': 'PRODAM S P',
 'txtTipoNatureza': 'C',
 'txtRazaoSocial': 'EMPRESA DE TECNOLOGIA DA INFORMACAO E COMUNICACAO DO MUNICIPIO DE SAO PAULO  PRODAMSP SA',
 'numCpfCnpj': '43076702000161'}

In [38]:
def get_contratos_by_credor(cnpj:str, ano:int=ANO_BASE)->JSONType:

    cnpj = clean_cnpj(cnpj)
    endpoint = 'contratos'

    return get_sof(endpoint, anoContrato=ano, numCpfCnpj=cnpj)

In [39]:
contratos_prodam = get_contratos_by_credor(prodam['numCpfCnpj'])

In [40]:
contratos_prodam = contratos_prodam[get_key_dados(contratos_prodam)]

In [41]:
len(contratos_prodam)

50

In [42]:
contratos_prodam[0]

{'txtDescricaoModalidade': 'Dispensa de Licitação',
 'codEmpresa': '07',
 'codTipoContratacao': '30',
 'valReajustes': 0.0,
 'codProcesso': '9310202400022280',
 'codModalidade': 6,
 'valAditamentos': 0.0,
 'codContrato': 18,
 'valLiquidado': 0.0,
 'codOrgao': '33',
 'valPago': 0.0,
 'valAnulacao': 0.0,
 'valEmpenhadoLiquido': 0.0,
 'valTotalEmpenhado': 3817.33,
 'anoContrato': 2024,
 'txtDescricaoOrgao': 'Agência Reg. de Serv. Públicos do Mun de São Paulo        ',
 'valAnuladoEmpenho': 3817.33,
 'valPrincipal': 45808.02,
 'txtTipoContratacao': 'Nota de Empenho para Fornecimento de Serviços',
 'txtObjetoContrato': 'Prestação dos serviços de Comunicação de Dados SD-WAN, pela Contratada, para o suporte aos negócios para a Agência Reguladora de Serviços Públicos do Município de São Paulo - SP REGULA,'}

In [43]:
df_prodam = pd.DataFrame(contratos_prodam)

In [44]:
df_prodam.head()

,txtDescricaoModalidade,codEmpresa,codTipoContratacao,valReajustes,codProcesso,codModalidade,valAditamentos,codContrato,valLiquidado,codOrgao,...,anoContrato,txtDescricaoOrgao,valAnuladoEmpenho,valPrincipal,txtTipoContratacao,txtObjetoContrato,datVigencia,datAssinaturaContrato,numOriginalContrato,datPublicacaoContrato
0,Dispensa de Licitação,07,30,0.0,9310202400022280,6,0.00,18,0.00,33,...,2024,Agência Reg. de Serv. Públicos do Mun de São P...,3817.33,45808.02,Nota de Empenho para Fornecimento de Serviços,Prestação dos serviços de Comunicação de Dados...,NaN,NaN,NaN,NaN
1,Dispensa de Licitação,19,2,0.0,000113512023,6,1065260.33,18,821316.83,10,...,2024,Tribunal de Contas do Município de São Paulo,2276.20,716940.33,Termo de Contrato - Serviços,Prestação de serviços de sustentação e melhori...,02/03/2026 00:00:00,03/03/2024 00:00:00,04/2024,05/03/2024 00:00:00
2,Não Aplicável,30,73,0.0,7810202400008736,8,0.00,35,207123.24,05,...,2024,São Paulo Urbanismo,41790.01,842387.00,Nota de Empenho para Fornecimento de Serviços ...,Prestação de Serviços de SUSTENTAÇÃO DE TIC pa...,15/07/2026 00:00:00,12/07/2024 00:00:00,7810.2024/0000873-6,16/07/2024 00:00:00
3,Dispensa de Licitação,19,2,0.0,000022682024,6,260850.30,52,215100.74,10,...,2024,Tribunal de Contas do Município de São Paulo,15430.01,260850.30,Termo de Contrato - Serviços,"Prestação de Serviços de Sustentação de TIC, r...",04/04/2026 00:00:00,04/04/2024 00:00:00,09/2024,05/04/2024 00:00:00
4,Dispensa de Licitação,85,2,0.0,8510202400006786,6,0.00,148,85654.16,85,...,2024,Fundação Theatro Municipal de São Paulo,8740.92,309523.09,Termo de Contrato - Serviços,Contratação de serviços de sustentação e melho...,27/10/2025 00:00:00,24/10/2024 00:00:00,094/FTMSP/2024,25/10/2024 00:00:00


In [45]:
df_prodam.columns

Index(['txtDescricaoModalidade', 'codEmpresa', 'codTipoContratacao',
       'valReajustes', 'codProcesso', 'codModalidade', 'valAditamentos',
       'codContrato', 'valLiquidado', 'codOrgao', 'valPago', 'valAnulacao',
       'valEmpenhadoLiquido', 'valTotalEmpenhado', 'anoContrato',
       'txtDescricaoOrgao', 'valAnuladoEmpenho', 'valPrincipal',
       'txtTipoContratacao', 'txtObjetoContrato', 'datVigencia',
       'datAssinaturaContrato', 'numOriginalContrato',
       'datPublicacaoContrato'],
      dtype='object')

In [46]:
cols_valores = [col for col in df_prodam.columns if col.startswith('val')]

In [47]:
df_prodam[cols_valores].dtypes

valReajustes           float64
valAditamentos         float64
valLiquidado           float64
valPago                float64
valAnulacao            float64
valEmpenhadoLiquido    float64
valTotalEmpenhado      float64
valAnuladoEmpenho      float64
valPrincipal           float64
dtype: object

In [48]:
assert (df_prodam[cols_valores].dtypes==float).all()

In [49]:
df_prodam[cols_valores].describe()

,valReajustes,valAditamentos,valLiquidado,valPago,valAnulacao,valEmpenhadoLiquido,valTotalEmpenhado,valAnuladoEmpenho,valPrincipal
count,50.00000,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01
mean,2029.16720,3.084040e+05,2.039053e+06,1.930142e+06,8.615340e+10,4.052518e+06,5.802262e+06,1.749744e+06,8.615903e+10
std,12981.39342,1.854287e+06,5.521347e+06,5.444506e+06,6.091966e+11,9.071614e+06,1.310308e+07,5.719006e+06,6.091957e+11
min,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.354770e+03
25%,0.00000,0.000000e+00,3.272576e+04,9.493380e+03,0.000000e+00,1.333724e+05,1.693489e+05,2.644650e+03,1.693489e+05
50%,0.00000,0.000000e+00,1.693489e+05,1.340924e+05,0.000000e+00,3.603815e+05,4.755086e+05,2.442311e+04,3.932707e+05
75%,0.00000,0.000000e+00,8.126508e+05,6.254909e+05,0.000000e+00,2.052676e+06,4.499964e+06,4.545394e+05,7.030733e+06
max,91809.85000,1.310027e+07,3.371202e+07,3.370902e+07,4.307670e+12,4.703095e+07,6.726700e+07,3.608543e+07,4.307670e+12


Aparentemente em alguns casos o numero do CNPJ da Prodam foi parar dentro dos valores

In [50]:
df_prodam[df_prodam['valAnulacao']>10**9]

,txtDescricaoModalidade,codEmpresa,codTipoContratacao,valReajustes,codProcesso,codModalidade,valAditamentos,codContrato,valLiquidado,codOrgao,...,anoContrato,txtDescricaoOrgao,valAnuladoEmpenho,valPrincipal,txtTipoContratacao,txtObjetoContrato,datVigencia,datAssinaturaContrato,numOriginalContrato,datPublicacaoContrato
36,Dispensa de Licitação,01,2,0.0,6074202400042915,6,0.0,30336,0.0,34,...,2024,Secretaria Municipal de Direitos Humanos e Cid...,481997.5,4.307670e+12,Termo de Contrato - Serviços,Prestação de Serviços de Acesso à Rede Corpora...,NaN,NaN,NaN,NaN


In [51]:
def if_larger_then_tresh_to_mean(df:pd.DataFrame, cols:List[str], tresh:float=10**9)->pd.DataFrame:

    df = df.copy()
    for col in cols:
        df[col] = df[col].apply(lambda x: x if x <  tresh else df[col].mean())

    return df
    

In [52]:
df_prodam = if_larger_then_tresh_to_mean(df_prodam, cols_valores)

In [53]:
df_prodam[cols_valores].describe()

,valReajustes,valAditamentos,valLiquidado,valPago,valAnulacao,valEmpenhadoLiquido,valTotalEmpenhado,valAnuladoEmpenho,valPrincipal
count,50.00000,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01
mean,2029.16720,3.084040e+05,2.039053e+06,1.930142e+06,1.723068e+09,4.052518e+06,5.802262e+06,1.749744e+06,1.728810e+09
std,12981.39342,1.854287e+06,5.521347e+06,5.444506e+06,1.218393e+10,9.071614e+06,1.310308e+07,5.719006e+06,1.218392e+10
min,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.354770e+03
25%,0.00000,0.000000e+00,3.272576e+04,9.493380e+03,0.000000e+00,1.333724e+05,1.693489e+05,2.644650e+03,1.693489e+05
50%,0.00000,0.000000e+00,1.693489e+05,1.340924e+05,0.000000e+00,3.603815e+05,4.755086e+05,2.442311e+04,3.932707e+05
75%,0.00000,0.000000e+00,8.126508e+05,6.254909e+05,0.000000e+00,2.052676e+06,4.499964e+06,4.545394e+05,7.030733e+06
max,91809.85000,1.310027e+07,3.371202e+07,3.370902e+07,8.615340e+10,4.703095e+07,6.726700e+07,3.608543e+07,8.615903e+10


In [54]:
df_prodam.sample(3)

,txtDescricaoModalidade,codEmpresa,codTipoContratacao,valReajustes,codProcesso,codModalidade,valAditamentos,codContrato,valLiquidado,codOrgao,...,anoContrato,txtDescricaoOrgao,valAnuladoEmpenho,valPrincipal,txtTipoContratacao,txtObjetoContrato,datVigencia,datAssinaturaContrato,numOriginalContrato,datPublicacaoContrato
28,Dispensa de Licitação,01,29,0.0,6023202400012350,6,0.0,21336,0.00,23,...,2024,Secretaria Municipal de Inovação e Tecnologia,0.00,87842.30,Termo de Contrato para Aquisição de Bens de Co...,Prestação de serviços de consultoria técnica e...,NaN,NaN,NaN,NaN
37,Dispensa de Licitação,01,2,0.0,6074202400042915,6,0.0,30364,0.00,34,...,2024,Secretaria Municipal de Direitos Humanos e Cid...,481997.50,1652562.80,Termo de Contrato - Serviços,Prestação de Serviços de Acesso à Rede Corpora...,NaN,NaN,NaN,NaN
49,Dispensa de Licitação,01,2,0.0,6068202400086545,6,0.0,35272,5336451.84,29,...,2024,Secretaria Municipal de Urbanismo e Licenciamento,568117.25,27415244.95,Termo de Contrato - Serviços,Prestação de Serviços para Sustentação de Infr...,31/10/2025 00:00:00,31/10/2024 00:00:00,17/2024/SMUL,04/11/2024 00:00:00


In [55]:
from utils.serialize_df import df_to_parquet

In [56]:
df_to_parquet(df_prodam, 'contratos_prodam.parquet')

'data/contratos_prodam.parquet'